In [ ]:
import pandas as pd
import torch
import os
import sys
sys.path.append("/mnt/raid/C1_ML_Analysis/source/us-famli-pl/src/")
from nets.classification import EffnetV2sSoft, FlyToClassification
import loaders.ultrasound_dataset as usd

import plotly.express as px
import SimpleITK as sitk

In [ ]:
mount_point = "/mnt/raid/C1_ML_Analysis/"
# model_fn = "train_output/classification/FlyToClassification/v0.4/epoch=34-val_loss=0.76.ckpt"
# model = FlyToClassification.load_from_checkpoint(model_fn)

model_fn = "train_output/classification/FlyToClassification/v0.4/epoch=34-val_loss=0.76.pt" 
model = torch.jit.load(model_fn)
model = model.eval().cuda()


In [ ]:
model_fn = "train_output/classification/FlyToClassification/v0.4/epoch=34-val_loss=0.76.pt" 
model = torch.jit.load(model_fn)
model = model.eval().cuda()

In [ ]:
# df_dxa = pd.read_csv(os.path.join(mount_point, 'CSV_files', '2025-08-20', 'dxa_instance_EFW_with_bw_dataset.csv'))
# df_dxa_dcm = df_dxa[df_dxa['file_path'].str.endswith('.dcm')]
# df_dxa_dcm.to_csv(os.path.join(mount_point, 'CSV_files', '2025-08-20', 'dxa_instance_EFW_with_bw_dataset_dcm.csv'), index=False)


In [ ]:
df_dxa = pd.read_csv(os.path.join(mount_point, 'CSV_files', '2025-08-20', 'dxa_instance_EFW_with_bw_dataset.csv'))
df_dxa.rename(columns={'file_path': 'file_path_orig'}, inplace=True)
df_dxa['file_path'] = df_dxa['file_path_orig'].str.replace('Groups/FAMLI/Restricted_access_data/Ultrasound/', '').str.replace('Groups/FAMLI/Deidentified_data/Ultrasound/', '').str.replace(f'DXA/', f'DXA_masked_resampled_256_spc075/').str.replace('.mp4', '.nrrd').str.replace('.dcm', '.nrrd')

In [ ]:
df_dxa.columns

In [ ]:
print(df_dxa.query('pid == "DXA-300-0127" and tag == "NL"')['file_path'])

In [ ]:
usd.TAGS_DICT

In [ ]:


img = sitk.ReadImage(os.path.join(mount_point, df_dxa.query('pid == "DXA-300-0127" and tag == "NL"')['file_path'].values[0]))
img_np = sitk.GetArrayFromImage(img)

px.imshow(img_np, animation_frame=0, color_continuous_scale='gray')

In [ ]:
img_np.shape

In [ ]:
img_t = torch.from_numpy(img_np[:,:,:,0]).unsqueeze(1).repeat(1, 3, 1, 1).unsqueeze(0).float().cuda()
tags = torch.tensor(usd.TAGS_DICT['NL']).to(torch.long).unsqueeze(0).cuda()

In [ ]:
img_t.shape

In [ ]:
with torch.no_grad():
    x_hat, x_s = model(img_t, tags)

In [ ]:
x_hat.argmax(dim=2)

In [ ]:

model_soft_fn = "train_output/classification/effnetv2s/ac_multi_label_ds_soft/v0.4/epoch=20-val_loss=0.92-v1.pt"
model = torch.jit.load(model_soft_fn)
model = model.eval().cuda()

img_t = torch.from_numpy(img_np[:,:,:,0]).unsqueeze(1).repeat(1, 3, 1, 1).float().cuda()

with torch.no_grad():
    x_hat = model(img_t)

In [ ]:
x_hat.argmax(dim=1)